# TF-IDF Project
* Este projeto busca a criação de um sistema de recomendação de filmes baseado em TF-iDF.
* O programa final deverá receber o nome de um filme e retornar uma quantidade n de filmes parecidos com ele.

## Carregamento dos Dados
* Vamos trabalhar apenas com a sinopse, gênero(s), keywords e cast dos filmes.

In [1]:
import pandas as pd

df_movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [2]:
df_credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [3]:
# Dataset com apenas os dados do projeto. 
df = pd.merge(df_movies, df_credits, left_on='id', right_on='movie_id')[['original_title', 'overview', 'genres', 'keywords', 'cast']]
df.head()

,original_title,overview,genres,keywords,cast
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."
2,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr..."
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba..."
4,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c..."


## Tratamento dos Dados
* Do que observei nas colunas de listas de dicionários, existe apenas uma chave que nos interessa em cada uma delas.
* Por isso, podemos transformar essas colunas no concatenado dos valores dessas chaves.

In [4]:
from typing import Dict, List

# Listando as colunas "dicionarizadas" e a chave de nosso interesse.
col_key = {col:'name' for col in ['genres', 'keywords', 'cast']}

def _concat_str(row_value:str, k:str)->str:
    '''
        Função que converterá a lista de dicionários em uma string com os valores da chave de interesse concatenados.
        
        Parâmetros
        ----------
        `row_value`: str
            Valor da linha de nosso interesse.
        `k`: str
            Nome da chave cujos valores substituirão a lista de dicionários.
            
        Retorna
        -------
        Uma string com os valores de interesse concatenados.
            
    '''
    l = eval(row_value)
    output_str = ''
    for d in l:
        output_str+=f'{d[k]}, '
    return output_str

def dict_to_str(df:pd.DataFrame=df, col_key: Dict[str, str]=col_key)->pd.DataFrame:
    '''
        Função que transformará as colunas "dicionarizadas".
        
        Parâmetros
        ----------
        `df`: pd.DataFrame
            DataFrame com os dados do projeto.
        `col_key`: Dict[str, str]
            Dicionário que menciona os nomes das colunas-alvo e chaves de interesse.
            
        Retorna
        -------
        DataFrame com as colunas transformadas em texto pleno.
    '''
    df = df.copy() # Não vamos aplicar as transformações in-place.
    for col, k in col_key.items():
        df[col] = df[col].apply(_concat_str, **{'k':k})
    return df
    
#df['cast'].map(lambda x: eval(x))
df = dict_to_str()
df.head()

,original_title,overview,genres,keywords,cast
0,Avatar,"In the 22nd century, a paraplegic Marine is di...","Action, Adventure, Fantasy, Science Fiction,","culture clash, future, space war, space colony...","Sam Worthington, Zoe Saldana, Sigourney Weaver..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","Adventure, Fantasy, Action,","ocean, drug abuse, exotic island, east india t...","Johnny Depp, Orlando Bloom, Keira Knightley, S..."
2,Spectre,A cryptic message from Bond’s past sends him o...,"Action, Adventure, Crime,","spy, based on novel, secret agent, sequel, mi6...","Daniel Craig, Christoph Waltz, Léa Seydoux, Ra..."
3,The Dark Knight Rises,Following the death of District Attorney Harve...,"Action, Crime, Drama, Thriller,","dc comics, crime fighter, terrorist, secret id...","Christian Bale, Michael Caine, Gary Oldman, An..."
4,John Carter,"John Carter is a war-weary, former military ca...","Action, Adventure, Science Fiction,","based on novel, mars, medallion, space travel,...","Taylor Kitsch, Lynn Collins, Samantha Morton, ..."


### Criação dos Textos para TF-IDF
* Tendo as todas as colunas como texto, podemos uni-las em uma única só com todos os seus conteúdos.
* Como nossa solução envolve apenas uma técnica Bag-of-Words, podemos apenas concatenar os valores das colunas, sem nos preocuparmos com a semântica.

In [5]:
# Concatenando os dados.
df['X'] = ''
for col in df.columns[1:]:
    df['X']+=(df[col]+' ')
    
# Removendo leading e trailing spaces.
df['X'] = df['X'].str.strip()

# Mantendo apenas as colunas de título e 'X'.
df = df[['original_title', 'X']]
df.head()

,original_title,X
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag, wordnet
from typing import List

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    
    @staticmethod
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
        
    def __call__(self, doc)->List[str]:
        tokens = word_tokenize(doc)
        tokens_tags = pos_tag(tokens)
        return [self.wnl.lemmatize(token, pos=self.get_wordnet_pos(pos)) for token, pos in tokens_tags]

In [7]:
# Invocando as stopwords da língua inglesa.
stop_words = stopwords.words('english')

In [8]:
# Criando nossa instância de TF-IDF
tf_idf = TfidfVectorizer(strip_accents='ascii', stop_words=stop_words)

# Gerando nossa bag-of-words.
X = tf_idf.fit_transform(df['X'].astype('str'))
X

<4803x53206 sparse matrix of type '<class 'numpy.float64'>'
	with 396284 stored elements in Compressed Sparse Row format>

In [9]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def query_movies(movie:str, k:int=5)->pd.DataFrame:
    global df
    idx_movie = df[df.original_title==movie].index[0]
    idx_similar = np.argsort(cosine_similarity(X)[idx_movie])[-k-1:-1]
    return df.iloc[idx_similar, 0].to_list()

query_movies('Avatar')

['Moonraker', 'Alien', 'Treasure Planet', 'Alien³', 'Aliens']

<p style='color:red'> X montado; Criar o mecanismo de queries usando np.where, np.argwhere</p>